# Assignment 2

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./05_src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model]() or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [1]:
# Load the libraries as required.
%load_ext dotenv
%dotenv


In [2]:
import pandas as pd
import os
import sys
from glob import glob
import numpy as np

sys.path.append(os.getenv('SRC_DIR'))

from utils.logger import get_logger
_logs = get_logger(__name__)

In [3]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = 0, names = columns))
fires_dt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


# Get X and Y

Create the features data frame and target data.


In [4]:
fires_dt

,coord_x,coord_y,month,day,ffmc,dmc,dc,isi,temp,rh,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,4,3,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44
513,2,4,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29
514,7,4,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16
515,1,4,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00


In [4]:
y = fires_dt['area']
X = fires_dt.drop(columns='area')

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss, cohen_kappa_score, f1_score, make_scorer, mean_squared_error

# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [6]:
# Define numerical and categorical columns based on the dataset
num_cols = ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']
cat_cols = ['month', 'day']

In [7]:
#Prerpocessor 1: Scale numeric variables, one-hot encode categorical
preproc1 = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [9]:
# Define which numeric colums to transform
log_transform_cols = ['dmc', 'dc', 'isi']
other_num_cols = list(set(num_cols) - set(log_transform_cols))

# Preprocessor 2: Apply log1p to selecte variables, sale all, one-hot encode categoricals
preproc2 = ColumnTransformer([
    ('log_num', Pipeline([
        ('log', FunctionTransformer(np.log1p, validate=True)),
        ('scaler', StandardScaler())
    ]), log_transform_cols),
    ('other_num', StandardScaler(), other_num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [10]:
# Pipeline A = preproc1 + baseline
pipeline_A = Pipeline([
    ('priprocessing', preproc1),
    ('regressor', Lasso())
    ])

param_gridA = {
    'regressor__alpha': [0.01, 0.1, 1, 10]
}

In [11]:
pipeline_A

Pipeline(steps=[('priprocessing',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['X', 'Y', 'FFMC', 'DMC',
                                                   'DC', 'ISI', 'temp', 'RH',
                                                   'wind', 'rain']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['month', 'day'])])),
                ('regressor', Lasso())])

In [18]:
# Pipeline B = preproc2 + baseline
pipeline_B = Pipeline([
    ('priprocessing', preproc2),
    ('regressor', Lasso())
])

param_gridB = {
    'regressor__alpha': [0.01, 0.1, 1.0, 10.0]
}

In [19]:
# Pipeline C = preproc1 + advanced model
pipeline_C = Pipeline([
    ('priprocessing', preproc1),
    ('regressor', RandomForestRegressor(random_state=42))
])

param_gridC = {
    'regressor__n_estimators': [50, 100],
    'regressor__max_depth': [5, None]
}

In [20]:
# Pipeline D = preproc2 + advanced model
pipeline_D = Pipeline([
    ('priprocessing', preproc2),
    ('regressor', RandomForestRegressor(random_state=42))
])

param_gridD = {
    'regressor__n_estimators': [50, 100],
    'regressor__max_depth': [5, None]
}

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [14]:
rmse_scorer = make_scorer(lambda y_true, y_pred: np.sqrt(np.mean((y_true - y_pred) ** 2)))

In [15]:
gridA = GridSearchCV(pipeline_A, param_gridA, cv=5, scoring=rmse_scorer)
gridA.fit(X, y)

print("pipeline_A (preproc1 + Lasso):")
print(f"Best RMSE: {gridA.best_score_:.4f}")
print(f"Best Params: {gridA.best_params_}\n")

pipeline_A (preproc1 + Lasso):
Best RMSE: 54.4010
Best Params: {'regressor__alpha': 0.01}



/opt/anaconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.134e+03, tolerance: 2.069e+02
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.387e+03, tolerance: 2.077e+02
  model = cd_fast.enet_coordinate_descent(
/opt/anaconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consi

In [21]:
print(pipeline_A.get_params().keys())

dict_keys(['memory', 'steps', 'transform_input', 'verbose', 'priprocessing', 'regressor', 'priprocessing__force_int_remainder_cols', 'priprocessing__n_jobs', 'priprocessing__remainder', 'priprocessing__sparse_threshold', 'priprocessing__transformer_weights', 'priprocessing__transformers', 'priprocessing__verbose', 'priprocessing__verbose_feature_names_out', 'priprocessing__num', 'priprocessing__cat', 'priprocessing__num__copy', 'priprocessing__num__with_mean', 'priprocessing__num__with_std', 'priprocessing__cat__categories', 'priprocessing__cat__drop', 'priprocessing__cat__dtype', 'priprocessing__cat__feature_name_combiner', 'priprocessing__cat__handle_unknown', 'priprocessing__cat__max_categories', 'priprocessing__cat__min_frequency', 'priprocessing__cat__sparse_output', 'regressor__alpha', 'regressor__copy_X', 'regressor__fit_intercept', 'regressor__max_iter', 'regressor__positive', 'regressor__precompute', 'regressor__random_state', 'regressor__selection', 'regressor__tol', 'regress

In [24]:
gridB = GridSearchCV(pipeline_B, param_gridB, cv=5, scoring=rmse_scorer)
gridB.fit(X, y)

print("pipeline_B (preproc2 + Lasso):")
print(f"Best RMSE: {gridB.best_score_:.4f}")
print(f"Best Params: {gridB.best_params_}\n")

pipeline_B (preproc2 + Lasso):
Best RMSE: 53.7735
Best Params: {'regressor__alpha': 0.01}



In [25]:
gridC = GridSearchCV(pipeline_C, param_gridC, cv=5, scoring=rmse_scorer)
gridC.fit(X, y)

print("pipeline_C (preproc1 + Lasso):")
print(f"Best RMSE: {gridC.best_score_:.4f}")
print(f"Best Params: {gridC.best_params_}\n")

pipeline_C (preproc1 + Lasso):
Best RMSE: 61.1001
Best Params: {'regressor__max_depth': None, 'regressor__n_estimators': 50}



In [26]:
gridD = GridSearchCV(pipeline_D, param_gridD, cv=5, scoring=rmse_scorer)
gridD.fit(X, y)

print("pipeline_D (preproc1 + Lasso):")
print(f"Best RMSE: {gridD.best_score_:.4f}")
print(f"Best Params: {gridD.best_params_}\n")

pipeline_D (preproc1 + Lasso):
Best RMSE: 60.4414
Best Params: {'regressor__max_depth': None, 'regressor__n_estimators': 100}



# Evaluate

+ Which model has the best performance?

# Export

+ Save the best performing model to a pickle file.

In [31]:
results = {
    'preproc1 + Lasso': gridA.best_score_,
    'preproc2 + Lasso': gridB.best_score_,
    'preproc1 + RF': gridC.best_score_,
    'preproc2 + RF': gridD.best_score_,
}

best_model_name = min(results, key=results.get)
print(f"best model: [best_model_name]with RMSE = {results[best_model_name]:.4f}")

best_model = {
    'preproc1 + Lasso': gridA,
    'preproc2 + Lasso': gridB,
    'preproc1 + RF': gridC,
    'preproc2 + RF': gridD,
}[best_model_name]

best model: [best_model_name]with RMSE = 53.7735


In [32]:
import pickle

with open('best_model.pkl', 'wb') as f:
    pickle.dump(best_model.best_estimator_, f)

print("Best model saved as 'best_model.pkl'")

Best model saved as 'best_model.pkl'


# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [33]:
import shap
import pickle

/opt/anaconda3/envs/dsi_participant/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [36]:
with open('best_model.pkl', 'wb') as f:
    pickle.dump(best_model.best_estimator_, f)

print("Best model saved as 'best_model.pkl")

Best model saved as 'best_model.pkl


*(Answer here.)*

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at the `help` channel. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.